In [238]:
import tensorflow as tf
import numpy as np
import os
import time

Download the data

#Ref: [Songs Dataset](https://raw.githubusercontent.com/donaldong/rnn-text-gen/master/data/songs.txt)

Read the data

First, look in the text:

In [239]:
# Read, then decode for py2 compat.
text = open('/content/songs.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 59782 characters


In [240]:
# Take a look at the first 250 characters in text
print(text[:301])

Dashing through the snow
On a one-horse open sleigh,
Over the fields we go,
Laughing all the way;
Bells on bob-tail ring,
Making spirits bright,
What fun it is to ride and sing
A sleighing song tonight
Jingle bells, jingle bells,
Jingle all the way!
O what fun it is to ride
In a one-horse open sleigh


In [241]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

77 unique characters


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [242]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

#example of how to map the first 13 charcters in the text to integers
print('{} ---- characters mapped to int ---> {}'.format(repr(text[:24]), text_as_int[:24]))

'Dashing through the snow' ---- characters mapped to int ---> [27 50 68 57 58 63 56  1 69 57 67 64 70 56 57  1 69 57 54  1 68 63 64 72]


The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [243]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(7):
  print(idx2char[i.numpy()])

D
a
s
h
i
n
g


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [244]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Dashing through the snow\nOn a one-horse open sleigh,\nOver the fields we go,\nLaughing all the way;\nBel'
'ls on bob-tail ring,\nMaking spirits bright,\nWhat fun it is to ride and sing\nA sleighing song tonight\n'
'Jingle bells, jingle bells,\nJingle all the way!\nO what fun it is to ride\nIn a one-horse open sleigh\n\n'
"A day or two ago,\nI thought I'd take a ride,\nAnd soon Miss Fanny Bright\nWas seated by my side;\nThe ho"
'rse was lean and lank;\nMisfortune seemed his lot;\nHe got into a drifted bank,\nAnd we, we got upsot.\nJ'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [245]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [246]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Dashing through the snow\nOn a one-horse open sleigh,\nOver the fields we go,\nLaughing all the way;\nBe'
Target data: 'ashing through the snow\nOn a one-horse open sleigh,\nOver the fields we go,\nLaughing all the way;\nBel'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [247]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [248]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [249]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [250]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [251]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 77) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [252]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (128, None, 256)          19712     
_________________________________________________________________
lstm_22 (LSTM)               (128, None, 1024)         5246976   
_________________________________________________________________
dropout_13 (Dropout)         (128, None, 1024)         0         
_________________________________________________________________
lstm_23 (LSTM)               (128, None, 1024)         8392704   
_________________________________________________________________
dropout_14 (Dropout)         (128, None, 1024)         0         
_________________________________________________________________
dense_20 (Dense)             (128, None, 77)           78925     
Total params: 13,738,317
Trainable params: 13,738,317
Non-trainable params: 0
_________________________________________

To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [253]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([63, 65, 69, 19, 47, 19,  3, 46, 51, 54,  8, 26, 48, 17, 58, 19, 65,
        1, 40, 20, 52,  4, 47,  4,  9, 49, 75, 46, 29, 20,  9, 34, 60, 72,
       62, 67, 64, 64, 56, 23, 26, 75, 19, 63, 30, 72, 48,  0, 30, 18, 43,
       43, 75, 17, 64, 12, 25,  5,  6, 40, 18, 21, 54, 62, 42, 64, 23, 22,
       35, 60, 72, 11, 21, 21,  0,  5, 26,  9, 59, 10, 10, 10, 50, 18, 76,
        9,  3,  6, 23, 65, 61, 32, 19, 36, 58, 18, 74, 53, 37, 70])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [254]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 77)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.343342


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [255]:
model.compile(optimizer='RMSprop', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [256]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [257]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
4/4 [==============================] - 6s 626ms/step - loss: 5.2774
Epoch 2/30
4/4 [==============================] - 3s 626ms/step - loss: 4.7481
Epoch 3/30
4/4 [==============================] - 2s 603ms/step - loss: 3.5552
Epoch 4/30
4/4 [==============================] - 3s 614ms/step - loss: 3.3111
Epoch 5/30
4/4 [==============================] - 2s 603ms/step - loss: 3.3113
Epoch 6/30
4/4 [==============================] - 3s 630ms/step - loss: 3.3129
Epoch 7/30
4/4 [==============================] - 2s 599ms/step - loss: 3.3147
Epoch 8/30
4/4 [==============================] - 3s 608ms/step - loss: 3.3154
Epoch 9/30
4/4 [==============================] - 3s 622ms/step - loss: 3.3167
Epoch 10/30
4/4 [==============================] - 3s 610ms/step - loss: 3.3236
Epoch 11/30
4/4 [==============================] - 3s 612ms/step - loss: 3.3002
Epoch 12/30
4/4 [==============================] - 2s 605ms/step - loss: 3.2984
Epoch 13/30
4/4 [==============================] 

KeyboardInterrupt: ignored

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Grandma:  "))